# Netzerkanalyse mit GitHub

Semesterarbeit 2

## EDS-Einführung in Data Science

Klasse: BSc INF-P-IN010, BE1, HS20/21<br>
Dozent: Dr. Tim vor der Brück<br>
Autor:  Sandro Bürki, Michael Friderich<br>
Datum:  09.10.2020<br>

## Einleitung

In der Entwicklung von Software steht für viele Firmen die effizienz an erster Stelle. Oftmals stellt sich hierbei die
Frage, ob sich der Einsatz von spezialisierten Fachpersonen im Vergleich zu Entwicklern mit breitem Know-How von
verschiedenen Projekten positiv auf die Effizienz im Team auswirkt. Viele Projektmethodiken oder Projektframeworks (wie
beispielsweise SCRUM) setzen dabei auf eine Fokussierung der Arbeitskraft, anstelle eines breiten Einsatzgebietes.

Diese Mentalität sollte sich somit in einer Netzwerkanalyse zeigen. Die Repositories von Firmen sollten primär von
spezifischen Personen gepflegt werden, welche sich nicht weiter stark an anderen Projekten betreffen. Mit dieser
Frage beschäftigt sich die vorliegende Arbeit.

Es gilt herauszufinden, wie stark sich die Repositories und ihre Contributors _segregieren_. Hierbei werden die
Centrality-Measurements miteinander verglichen. Sollte sich unsere These bestätigen, dass diese Segregierung eintritt,
sollten folgende Eigenschaften gelten:
1. Die Anzahl an Edges liegt nicht weit über der Anzahl an Contributors.
2. Die Betweenness-Centrality der meisten Benutzergeht ist 0.
3. Die Betweenness-Centrality der restlichen Benutzer geht gegen 0.

## Vorgehen

Der Code ist weitgehend in zwei Teile aufgeteilt. Zuerst werden die Daten von GitHub abgerufen. Anschliessend wird von
den Daten ein Backup in ein .json-File erstellt, da die Abfrage über viele Repositories viel Zeit in anspruch
nehmen kann, und somit die wiederverwendbarkeit nach einem Neustart des Jupyter-Notebooks vereinfacht wird.

Im zweiten Teil des Codes werden anhand der Daten findings gezogen.

### Daten abrufen

Zuerst werden die Informationen zu einer Firma aus GitHub geladen. Hierbei muss die entsprechende Firma und der
Access-Token gesetzt werden.

In [4]:
# HTTP request to GitHub's API
import json
from math import ceil
import requests
from IPython.core.display import display
from ipywidgets import HTML

ACCESS_TOKEN = '<INSERT-ACCESS-TOKEN-HERE>'

organisationName = "Google"

organisationUrlTemplate = 'https://api.github.com/orgs/{}'
organisationUrl = organisationUrlTemplate.format(organisationName)

response = requests.get(organisationUrl)
organisation = response.json()

organisationRepoCount = organisation["public_repos"]
print("number of repos found: {}".format(organisationRepoCount))

number of repos found: 1923


Anschliessend werden alle Repositories der Firma geladen. Da jeweils nur 100 Repos pro Aufruf möglich sind, werden sie
in entsprechenden Teilen zusammengefügt.

In [2]:
pageSize = 100

repositories = []
for page in range(0, ceil(organisationRepoCount / pageSize) + 1):

    organisationReposUrlTemplate = '{}?type=source&per_page=100&page={}&access_token={}'
    organisationReposUrl = organisationReposUrlTemplate.format(organisation["repos_url"], page, ACCESS_TOKEN)
    response = requests.get(organisationReposUrl)
    repositories.extend(response.json())
    print("Repository page #{} loaded. ({} / {})".format(page, len(repositories), organisationRepoCount))

Repository page #0 loaded. (2 / 1923)
Repository page #1 loaded. (4 / 1923)
Repository page #2 loaded. (6 / 1923)
Repository page #3 loaded. (8 / 1923)
Repository page #4 loaded. (10 / 1923)
Repository page #5 loaded. (12 / 1923)
Repository page #6 loaded. (14 / 1923)
Repository page #7 loaded. (16 / 1923)
Repository page #8 loaded. (18 / 1923)
Repository page #9 loaded. (20 / 1923)
Repository page #10 loaded. (22 / 1923)
Repository page #11 loaded. (24 / 1923)
Repository page #12 loaded. (26 / 1923)
Repository page #13 loaded. (28 / 1923)
Repository page #14 loaded. (30 / 1923)
Repository page #15 loaded. (32 / 1923)
Repository page #16 loaded. (34 / 1923)
Repository page #17 loaded. (36 / 1923)
Repository page #18 loaded. (38 / 1923)
Repository page #19 loaded. (40 / 1923)
Repository page #20 loaded. (42 / 1923)


Zu den Repositories können nun die Informationen der Contributors geladen werden. Da dies je nach Firma viele
Repositories sein können (im Beispiel von Google ca. 1900), kann dies einige Zeit in Anspruch nehmen.

In [9]:
print("Loading contributors of repositories. Depending on the size, this might take a while.")
numberOfRepositories = len(repositories)

for index, repository in enumerate(repositories):
    contributorUrlTemplate = "{}?per_page=100&page={}&access_token={}"
    repository["contributors"] = []
    page = 0
    while True:
        response = requests.get(contributorUrlTemplate.format(repository["contributors_url"], page, ACCESS_TOKEN))
        if response.status_code == 204:
            contributors = []
        else:
            contributors = response.json()

        page += 1
        if len(contributors) != 100:
            break

    if "message" in contributors:
        contributors = []

    repository['contributors'].extend(contributors)
    print("Contributors for repo #{} of {} loaded".format(index + 1, numberOfRepositories))

Loading contributors of repositories. Depending on the size, this might take a while.


TypeError: 'str' object does not support item assignment

Für die einfache Wiederverwertung können hier BackUps der geladenen Daten erstellt werden.

In [ ]:
with open('repositories_backup.json', 'w') as fp:
    json.dump(repositories, fp)

In [ ]:
import json

with open('repositories_backup.json', 'r') as fp:
    repositories = json.load(fp)

GitHub bietet bei einem kostenlosen Plan nicht für beliebige Grössen der Repositories an, alle Contributors zu laden,
weshalb hier die Fehlermeldungen einiger Repos entfernt werden.

In [12]:
for repo in repositories:
    if "message" in repo["contributors"]:
        repo["contributors"] = []

TypeError: string indices must be integers

Und um schlussendlich die Anzahl an Knoten an die Aufgabenstellung anzupassen, werden hier nur Repositories
berücksichtigt, welche über mehr als 80 Contributors umfassen. Die ungefilterte Liste würde ansonsten für Google ca.
12000 Knoten erstellen, was den Ramen dieser Arbeit sprengen würde.

In [13]:
largeRepos = []
for repo in repositories:
    if len(repo["contributors"]) > 80:
        largeRepos.append(repo)

TypeError: string indices must be integers

### Graphen erstellen

Im Folgenden wird ein Graph der Daten erstellt. Ebenfalls wird daraus ein .gefx-File generiert, welches die
Visualisierung in Gephi ermöglicht.

In [14]:
import networkx as nx

# Create a directed graph
g = nx.Graph()
for repository in largeRepos:
    g.add_node(repository["name"]+'(Repo)', type='Repo', color="green")
    contributors = repository["contributors"]
    for contributor in contributors:
        g.add_node(contributor["login"]+'(Contributor)', type='Contributor', color="pink")
        g.add_edge(contributor["login"], repository["name"]+'(Repo)', type='contribution', weight=contributor["contributions"])

# Write to File (open with Gephi)
nx.write_gexf(g, 'graph.gexf')

### Messwerte berechnen

Nachfolgend werden die gestellten Fragen mathematisch beantwortet

In [15]:
from operator import itemgetter

# Check findings mathematically
bc = sorted(nx.betweenness_centrality(g).items(), key=itemgetter(1), reverse=True)

# 1. property
numberOfContributors = 0
for node in bc:
    if "Contributor" in node[0]:
        numberOfContributors += 1

print('1. Die Anzahl an Edges liegt nicht weit über der Anzahl an Contributors.')
print('# of edges: {}'.format(nx.number_of_edges(g)))
print('# of contributors: {}'.format(numberOfContributors))

# 2. property

numberOfContributorsWithZeroBetweenness = 0
for node in bc:
    if "Contributor" in node[0] and node[1] == 0:
        numberOfContributorsWithZeroBetweenness += 1

print()
print('2. Die Betweenness-Centrality der meisten Benutzer ist 0.')
print('# of contributors with 0 bewteenness-centrality: {} (of {})'.format(numberOfContributorsWithZeroBetweenness, numberOfContributors))

print()
print('3. Die Betweenness-Centrality der restlichen Benutzer geht gegen 0.')
print(' -> ergibt sich aus 2.')

1. Die Anzahl an Edges liegt nicht weit über der Anzahl an Contributors.
# of edges: 0
# of contributors: 0

2. Die Betweenness-Centrality der meisten Benutzer ist 0.
# of contributors with 0 bewteenness-centrality: 0 (of 0)

3. Die Betweenness-Centrality der restlichen Benutzer geht gegen 0.
 -> ergibt sich aus 2.


## Findings

Die Resultate bestätigt unsere These, zumindest im Fall von Google. Insbesondere in der Visualisierung des Graphen in
Gephi lassen sich die Repository-Inseln gut erkennen, und es gibt nur einzelne Benutzer, welche an mehreren Projekten
gleichzeitig arbeiten. Für diese Visualisierung wurde der Layout-Algorithmus _ForceAtlas 2_ verwendet.

In [16]:
from IPython.display import HTML
from IPython.core.display import display

display(HTML('<img src="images/gephi.png" width="600px" style="margin: auto" />'))

###### Abbildung 1 - Visualisierung in Gephi
---
## Abbildungsverzeichnis
Abbildung 1: Friderich A., Bürki S. (2020)

## Literaturverzeichnis

[1] Russell, Matthew A. / Mikhail Klassen (2019): Mining the Social Web: Data Mining Facebook, Twitter, LinkedIn, Instagram, GitHub, and More, 3. Aufl., Sebastopol, USA, California: O’Reilly Media.